In [1]:
import pickle
import numpy as np
import random
from sklearn.neural_network import MLPClassifier
from tqdm.notebook import tqdm

random.seed(42)

In [2]:
# x_path = ''
# x_path = 'C:/Users/Carrt/OneDrive/Code/Motion Privacy/Defense Models/Mean Skeleton/X_FileNameKey_SingleActor.pkl'
# x_path = 'C:\\Users\\Carrt\\OneDrive\\Code\\Motion Privacy\\External Repositories\\Skeleton-anonymization\\X_resnet_file.pkl'
x_path = 'C:\\Users\\Carrt\\OneDrive\\Code\\Motion Privacy\\External Repositories\\Skeleton-anonymization\\X_unet_file.pkl'

with open(x_path, 'rb') as f:
    X = pickle.load(f)

In [3]:
# No overlap
# val_files = 'C:\\Users\\Carrt\\OneDrive\\Code\\Motion Privacy\\Linkage Attack\\SGN Based Linkage Attack\\data\\ntu120_no_dupe_actors.pkl'

# Overlap
# val_files = 'C:\\Users\\Carrt\\OneDrive\\Code\\Motion Privacy\\Linkage Attack\\SGN Based Linkage Attack\\data\\ntu120.pkl'

# Testing Data
val_files = 'C:\\Users\\Carrt\\OneDrive\\Code\\Motion Privacy\\Skeleton Info\\File Sorting\\ntu60.pkl'

# All Data
# val_files = 'data/ntu120.pkl'

with open(val_files, 'rb') as f:
    files_ = pickle.load(f)

to_del = []
for file in X:
    if file not in files_:
        to_del.append(file)

for file in to_del:
    del X[file]

In [4]:
def anonymizer_to_sgn(t, max_frames=300):
    # (300, 150)
    # [x:0,y:1,z:2][300][joints][actors]
    xyz,frames,joints,actors = t.shape
    # transpose to make loop simple
    # [frame][actors][joints][xyz]
    t = t.transpose(1,3,2,0)
    # make empty array
    frames = []
    
    joints_per_frame = xyz*joints*actors
    
    # crazy loop
    for frame in t:
        f = []
        for actor in frame:
            for joint in actor:
                for xyz in joint:
                    f.append(xyz)
        
        # Pad 0's to 150 joints (2 actors)
        if len(f) < joints_per_frame:
            f = np.pad(f, (0, joints_per_frame-len(f)), 'constant')
            
        frames.append(f)
        
    # to numpy array
    X = np.array(frames, dtype=np.float32)
    
    if X.shape[0] < max_frames:
        X = np.pad(X, ((0, max_frames-X.shape[0]), (0, 0)), 'constant')
        
    return X

if 'External Repositories' in x_path:
    X = {k: v[0] for k, v in X.items()}

    for file in X:
        X[file] = anonymizer_to_sgn(X[file])[:50,:75]

In [5]:
test_x = {}

# [{actor1: [frame], actor2: [frame]}]
test = [] 
same_samples_per_actor = 250
diff_samples_per_actor = 250

actor_data = {}
for file in X:
    actor = int(file[9:12])
    action = int(file[17:20])

    if actor not in actor_data:
        actor_data[actor] = []
    if len(X[file]) == 0:
        continue
    actor_data[actor].append(X[file])

actor_keys = list(actor_data.keys())

for actor in tqdm(actor_keys):
    for i in range(same_samples_per_actor):
        # Randomly select 2 frames from the same actor
        video1 = random.choice(actor_data[actor])
        video2 = random.choice(actor_data[actor])

        # Pad or clip video to 50 frames
        if len(video1) < 50: video1 = np.pad(video1, ((0, 50 - len(video1)), (0, 0)), 'constant')
        elif len(video1) > 50: video1 = video1[:50]
        if len(video2) < 50: video2 = np.pad(video2, ((0, 50 - len(video2)), (0, 0)), 'constant')
        elif len(video2) > 50: video2 = video2[:50]

        # Flatten Video
        video1 = video1.flatten()
        video2 = video2.flatten()

        # Add to test
        test.append({actor: np.array([video1, video2]), 'is_same': True})

    for i in range(diff_samples_per_actor):
        # Randomly select 2 frames from different actors
        while True: # Make sure the 2 frames are not from the same actor
            random_actor = random.choice(actor_keys)
            if random_actor != actor:
                video1 = random.choice(actor_data[actor])
                video2 = random.choice(actor_data[random_actor])
                
                # Pad or clip video to 50 frames
                if len(video1) < 50: video1 = np.pad(video1, ((0, 50 - len(video1)), (0, 0)), 'constant')
                elif len(video1) > 50: video1 = video1[:50]
                if len(video2) < 50: video2 = np.pad(video2, ((0, 50 - len(video2)), (0, 0)), 'constant')
                elif len(video2) > 50: video2 = video2[:50]
                
                # Flatten Video
                video1 = video1.flatten()
                video2 = video2.flatten()

                # Add to test
                test.append({actor: video1, random_actor: video2, 'is_same': False})
                break

test = np.array(test)

  0%|          | 0/40 [00:00<?, ?it/s]

In [6]:
X_test = []
y_test = []

for sample in test:
    if sample['is_same']:
        for actor in sample:
            if actor != 'is_same':
                X_test.append(np.concatenate([sample[actor][0], sample[actor][1]]))
        y_test.append(1)
    else:
        temp = []
        for actor in sample:
            if actor != 'is_same':
                temp.append(sample[actor])
        X_test.append(np.concatenate(temp))
        y_test.append(0)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [7]:
with open('mlp_classifier.pkl', 'rb') as f:
    clf = pickle.load(f)

In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

y_pred = clf.predict(X_test)
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1 Score: ', f1_score(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))

Accuracy:  0.5068
Precision:  0.5080340264650284
Recall:  0.43
F1 Score:  0.46577123050259966
Confusion Matrix:
 [[5836 4164]
 [5700 4300]]
